In [153]:
import sys
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr, kendalltau
import plotly.express as px
import matplotlib.pyplot as plt
# adding Folder_2 to the system path
sys.path.insert(0, r'C:\Users\tomer\Desktop\year3\sem B\workshop_microbiome\code')

import imports

In [154]:
data_df = pd.read_csv(r"C:\Users\tomer\Desktop\year3\sem B\workshop_microbiome\train_data.csv", index_col="sample")
metadata_df = pd.read_csv(r"C:\Users\tomer\Desktop\year3\sem B\workshop_microbiome\train_metadata.csv",  index_col="sample")
metadata_df["collection_date_days"] = (pd.to_datetime(metadata_df['collection_date']) - pd.Timestamp('1970-01-01')).dt.days

In [155]:
microbiom_with_sample_data = imports.clr_normalization( (data_df)).join(metadata_df[["baboon_id", "collection_date_days","collection_date"]], how="inner")
microbiom_with_sample_data = microbiom_with_sample_data.sort_values(["baboon_id", "collection_date_days"])
microbiom_with_sample_data["time_diff"] = microbiom_with_sample_data.groupby("baboon_id")["collection_date_days"].diff()
microbiom_with_sample_data["time_diff_shift"] = microbiom_with_sample_data["time_diff"].shift(-1)


In [156]:
cur = microbiom_with_sample_data[microbiom_with_sample_data["time_diff"] <=30]
prev = microbiom_with_sample_data[microbiom_with_sample_data["time_diff_shift"] <=30]

In [157]:
len(cur)

3600

In [158]:
def corr_per_baboon(cur, prev, id):
    cur_filtered = cur[cur["baboon_id"] == id].drop(columns=["baboon_id", "collection_date_days", "collection_date", "time_diff", "time_diff_shift"])
    prev_filtered = prev[prev["baboon_id"] == id].drop(columns=["baboon_id", "collection_date_days", "collection_date", "time_diff", "time_diff_shift"])
    corr_matrix = []
    for genus_cur in cur_filtered.columns:
        corr = []
        for genus_prev in prev_filtered.columns:
            corr.append(spearmanr(cur_filtered[genus_cur], (prev_filtered[genus_prev]))[0])
        corr_matrix.append(corr)

    return np.array(corr_matrix)

In [165]:
corr = np.zeros((61,61))
for baboon_id in microbiom_with_sample_data["baboon_id"].unique():
    corr += corr_per_baboon(cur, prev, baboon_id)


In [167]:
px.imshow(abs(corr/80), color_continuous_scale='viridis', width=800, height=800)


In [147]:
microbiom_filter_30 = microbiom_with_sample_data[microbiom_with_sample_data["time_diff"]<=30]

In [148]:
microbiom_filter_30["pervius_sample"] = microbiom_filter_30.groupby("baboon_id")["collection_date"].shift(1)

C:\Users\tomer\AppData\Local\Temp\ipykernel_2904\336046070.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

